# InvarLock: Programmatic Python API

[![Open in Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/invarlock/invarlock/blob/main/notebooks/invarlock_python_api.ipynb)

**Purpose:** Run the InvarLock pipeline from Python code (not the CLI).
**Runtime:** ~10 minutes
**Requires:** `invarlock[hf]` (adapters + eval + guards + edits)

This notebook demonstrates:

- Loading an HF model + tokenizer
- Building calibration windows with a dataset provider
- Running `CoreRunner.execute(...)`
- Generating a certificate with `invarlock.assurance.make_certificate` (using a small report adapter)


In [ ]:
!pip -q install "invarlock[hf]"
import sys
import invarlock

print("Python:", sys.version.split()[0])
print("InvarLock:", invarlock.__version__)


In [ ]:
!invarlock doctor --json || true


## Build calibration windows (synthetic provider)


In [ ]:
import os
from datetime import datetime
from pathlib import Path

os.environ.setdefault('INVARLOCK_ALLOW_NETWORK', '1')

from transformers import AutoTokenizer

from invarlock.eval.data import get_provider

MODEL_ID = 'sshleifer/tiny-gpt2'
SEQ_LEN = 128
PREVIEW_N = 16
FINAL_N = 16

tokenizer = AutoTokenizer.from_pretrained(MODEL_ID)
provider = get_provider('synthetic')
preview, final = provider.windows(
    tokenizer,
    preview_n=PREVIEW_N,
    final_n=FINAL_N,
    seq_len=SEQ_LEN,
    stride=SEQ_LEN,
)

calibration_data = [
    {'input_ids': ids, 'attention_mask': mask}
    for ids, mask in zip(
        preview.input_ids + final.input_ids,
        preview.attention_masks + final.attention_masks,
        strict=False,
    )
]

print('calibration windows:', len(calibration_data))


## Run baseline (no-op) and subject (quant_rtn)


In [ ]:
import json

from invarlock.adapters import HF_Causal_Auto_Adapter
from invarlock.assurance import make_certificate, render_certificate_markdown, validate_certificate
from invarlock.core.api import RunConfig
from invarlock.core.runner import CoreRunner
from invarlock.edits import NoopEdit, RTNQuantEdit
from invarlock.guards import InvariantsGuard, SpectralGuard
from invarlock.reporting.report_types import create_empty_report

adapter = HF_Causal_Auto_Adapter()
guards = [InvariantsGuard(), SpectralGuard()]

runner = CoreRunner()
config = RunConfig(device='auto', context={'profile': 'dev', 'tier': 'balanced'})

baseline_model = adapter.load_model(MODEL_ID, device='auto')
baseline_core = runner.execute(
    model=baseline_model,
    adapter=adapter,
    edit=NoopEdit(),
    guards=guards,
    config=config,
    calibration_data=calibration_data,
    preview_n=PREVIEW_N,
    final_n=FINAL_N,
)

subject_model = adapter.load_model(MODEL_ID, device='auto')
subject_core = runner.execute(
    model=subject_model,
    adapter=adapter,
    edit=RTNQuantEdit(bitwidth=8, per_channel=True, group_size=128, clamp_ratio=0.005),
    guards=guards,
    config=config,
    calibration_data=calibration_data,
    preview_n=PREVIEW_N,
    final_n=FINAL_N,
)

def core_to_run_report(core, *, edit_name: str) -> dict:
    report = create_empty_report()
    report['meta'].update({
        'model_id': MODEL_ID,
        'adapter': getattr(adapter, 'name', 'auto'),
        'device': 'auto',
        'commit': '',
        'seed': 42,
        'ts': datetime.now().isoformat(),
        'auto': None,
    })
    report['data'].update({
        'dataset': 'synthetic',
        'split': 'na',
        'seq_len': SEQ_LEN,
        'stride': SEQ_LEN,
        'preview_n': PREVIEW_N,
        'final_n': FINAL_N,
    })
    edit_meta = getattr(core, 'edit', {})
    if isinstance(edit_meta, dict):
        report['edit']['plan_digest'] = str(edit_meta.get('plan_digest', ''))
        if isinstance(edit_meta.get('deltas'), dict):
            report['edit']['deltas'] = edit_meta['deltas']
    report['edit']['name'] = edit_name
    metrics = getattr(core, 'metrics', {})
    if isinstance(metrics, dict):
        report['metrics'].update(metrics)
    ew = getattr(core, 'evaluation_windows', None)
    if isinstance(ew, dict) and ew:
        report['evaluation_windows'] = ew
    guards_map = getattr(core, 'guards', {})
    if isinstance(guards_map, dict):
        for name, result in guards_map.items():
            if not isinstance(result, dict):
                continue
            report['guards'].append({
                'name': name,
                'policy': result.get('policy', {}),
                'metrics': result.get('metrics', {}),
                'actions': result.get('actions', []),
                'violations': result.get('violations', []),
            })
    report['flags'].update({'guard_recovered': False, 'rollback_reason': None})
    report['artifacts'].update({'events_path': '', 'logs_path': '', 'checkpoint_path': None})
    return report

baseline_report = core_to_run_report(baseline_core, edit_name='noop')
subject_report = core_to_run_report(subject_core, edit_name='quant_rtn')

certificate = make_certificate(subject_report, baseline_report)
validate_certificate(certificate)
print('overall_pass:', certificate.get('validation', {}).get('overall_pass'))

Path('reports/python_api').mkdir(parents=True, exist_ok=True)
Path('reports/python_api/evaluation.cert.json').write_text(json.dumps(certificate, indent=2), encoding='utf-8')

print(render_certificate_markdown(certificate)[:800])


## Verify the programmatically-generated certificate


In [ ]:
%%bash
invarlock verify --json reports/python_api/evaluation.cert.json


## Related docs

- `docs/reference/api-guide.md`
- `docs/reference/programmatic-quickstart.md`
